<a href="https://colab.research.google.com/github/mansueli/Supa-Migrate/blob/main/Backup_Project_%26_Storage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Supabase](https://raw.githubusercontent.com/supabase/supabase/master/packages/common/assets/images/supabase-logo-wordmark--light.svg)



## Setting the Environment variables:

In [ ]:
OLD_DB_URL='https://old_project.supabase.co'
OLD_DB_PASS='secret_password_123'
OLD_SERVICE_KEY = 'eyJ0000J9.eyJQ.oPyK-LSECRET-aC1I'

## Installing PSQL, Downloading the scripts:

In [ ]:
!sudo sh -c 'echo "deb http://apt.postgresql.org/pub/repos/apt $(lsb_release -cs)-pgdg main" > /etc/apt/sources.list.d/pgdg.list'
!wget --quiet -O - https://www.postgresql.org/media/keys/ACCC4CF8.asc | sudo apt-key add -
!sudo apt-get update &>log
!sudo apt-get -y install postgresql &>log
!wget https://raw.githubusercontent.com/mansueli/Supa-Migrate/main/backup_database.sh &>log
!pip install supabase &>log

## Running migration & moving storage objects:

In [ ]:
from supabase import create_client
from google.colab import files
import os
filedata = ''
with open('backup_database.sh', 'r') as file :
  filedata = file.read()
  filedata = filedata.replace('db.old_project_ref.supabase.co', OLD_DB_URL.replace("https://","db."))
  filedata = filedata.replace('secret_password_here', OLD_DB_PASS)

with open('backup_database.sh', 'w') as file:
  file.write(filedata)

#Run the database migration script:
!bash ./backup_database.sh &>log

## Downloading Storage objects:

In [ ]:
#creating the clients for the old & new projects
old_client = create_client(OLD_DB_URL, OLD_SERVICE_KEY)

#Create all buckets
# Function to recursively copy files and directories
def copy_files(old_client, bucket_name, path):
    objects = old_client.storage.from_(bucket_name).list(path=path)

    for obj in objects:
        if obj['metadata'] is None:  # It's a directory
            subdirectory = os.path.join(path, obj['name'])
            # Create the directory if it doesn't exist
            if not os.path.exists(subdirectory):
                os.makedirs(subdirectory)
            copy_files(old_client, bucket_name, subdirectory)
        else:
            try:
                print(f"Downloading {path}/{obj['name']}")
                # Download file
                download_path = os.path.join(path, obj['name'])
                with open(download_path, 'wb') as f:
                    res = old_client.storage.from_(bucket_name).download(f"{path}/{obj['name']}")
                    f.write(res)
            except Exception as e:
                print("Error: ", e)

#Create all buckets
buckets = old_client.storage.list_buckets()
for bucket in buckets:
    print("Copying objects from "+bucket.name)
    objects = old_client.storage.from_(bucket.name).list()
    try:
      copy_files(old_client, bucket.name, "")
    except Exception as e:
      print("\n---\nAn error happened" + str(e))
    finally:
      copy_files(old_client, bucket.name, "")
